This notebook is about determining a region around which we will taking account of the FISH, we don't want to detect them in regions that are very far away from the cell bodies.

- We proceed by applying a morphological dilation of a certain extent around the nuclei.


23/05/25     Jacques Bourg @ Florian Muller lab. Institut Pasteur.
<img src="./HOX_pipeline.png" alt="LNP pipeline" width="1200" height="477">

In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
import skimage.io as io
import napari

import ipywidgets as widgets
from IPython.display import display

In [ ]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))
sys.path.append(str(base_dir / "detection_fish"))
sys.path.append(str(base_dir / "segmentation"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.file_handling import FileProcessor
from segmentation.refine_seg import Segmentation

tk  = Track()
sg  = Segmentation()
fp  = FileProcessor()

In [ ]:
var = str(Path('../Analysis'))
batch_folders = os.listdir(var)
dropdown = widgets.Dropdown(options=batch_folders, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown)

In [ ]:
n         = np.where(np.array(batch_folders) == dropdown.value)[0][0]
file_path = str(Path(var) / Path(batch_folders[n]) / Path(batch_folders[n] +'.json'))
constants = tk.load_json(file_path)
batch_name= constants['BATCH_NAME']; print(batch_name)

In [ ]:
modalities = constants['MODALITIES']
dropdown2 = widgets.Dropdown(options=modalities, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown2)

In [ ]:
n2  = np.where(np.array(modalities) == dropdown2.value)[0][0]
modality = modalities[n2]
print(modality)

In [ ]:
batch_mip                     = constants[f'BATCH_{modality}_DAPI_NUCLEI_MIP']

mask_cells_cleaned_list_path  = constants[f"MASK_{modality}_DAPI_CELL_CLEANED_LIST_PATH"]
dict_mask_cells_clean         = fp.load_masks_distributed_files(mask_cells_cleaned_list_path)

mask_cells_list_path          = constants[f"MASK_{modality}_DAPI_CELL_LIST_PATH"]
dict_mask_cells               = fp.load_masks_distributed_files(mask_cells_list_path)

dict_mask_cells_to_rem_path   = constants[f"MASK_{modality}_DAPI_NUCLEI_REMOVED_LABELS_LIST_PATH"]
dict_mask_cells_to_rem        = np.load(dict_mask_cells_to_rem_path,allow_pickle=True)[()]

In [ ]:
fish_area_folder    = Path(f"../Analysis/{batch_name}/{modality}/DAPI/fish_area")    
if not fish_area_folder.exists():
    fish_area_folder.mkdir(parents=True)
mask_fish_area_path = Path(f"../Analysis/{batch_name}/{modality}/DAPI/fish_area/{batch_name}_masks_fish_area.npy")

#### AUTOMATIC DILATION (dilating cell masks)

In [ ]:
distance_dilation_fish_area = 40     # dilate the cells ROI by this quantity (in pixels)
 
dict_mask_fish  = {}
viewer6  = napari.Viewer(title="FISH AREA")
counter  = 0
for ind, file in enumerate(batch_mip):
    file            = Path(file)
    file_name       = str(file.stem)
    im              = io.imread(file)
    basename        = '_'.join(Path(file).stem.split('_')[:-3])  # the dict key of disk_mask_fish should be the basename (independent of which channel was used to compute the fish area) 
    
    masks_cells_c   = dict_mask_cells_clean[basename]                                                                 
    masks_b         = (masks_cells_c>0)*1  # bin masks
    masks_fish      = sg.dilate_labels(masks_b, distance=distance_dilation_fish_area)
    mask_fish_clean = sg.remove_labels_from_fishmask(masks_fish, dict_mask_cells[basename], dict_mask_cells_to_rem[basename])      # remove from the fish area, the roi of the cells that we discarded before 

    val             = np.percentile(im, 99)
    viewer6.add_image(im, rgb=False, name=f"MIP  {file_name}", contrast_limits=[0, val])
    fish_layer   = viewer6.add_labels(masks_fish, name=f"FISH AREA {file_name}", opacity=.3, blending='translucent')
    fish_layer_c = viewer6.add_labels(mask_fish_clean, name=f"FISH AREA CLEAN {file_name}", opacity=.3, blending='translucent')

    if counter != 0:
        viewer6.layers[f"MIP  {file_name}"].visible             = False
        viewer6.layers[f"FISH AREA {file_name}"].visible        = False 
        viewer6.layers[f"FISH AREA CLEAN {file_name}"].visible  = False 

    counter +=1
    dict_mask_fish[basename] = mask_fish_clean
 
fp.save_masks_distributed_files(mask_fish_area_path, dict_mask_fish)

##### turn important lower case variables into uppercase for tracking, create those variables names with this content: useful for experiment tracking

In [ ]:
exec(f"MASK_{modality}_FISH_AREA_PATH = mask_fish_area_path", globals())
exec(f"DISTANCE_DILATION_{modality}_FISH_AREA = distance_dilation_fish_area", globals())

In [ ]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, batch_name, folder_path = Path(f"../Analysis/{batch_name}"))